In [ ]:
import pandas as pd

: 

In [ ]:
df = pd.read_csv("spotify_millsongdata.csv")

: 

In [74]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [75]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [76]:
df.shape

(57650, 4)

In [77]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [78]:
df = df.sample(5000).drop('link', axis = 1).reset_index(drop = True)

In [79]:
df.head(10)

,artist,song,text
0,Hank Snow,El Paso,Out in the west Texas town of El Paso I fell i...
1,Drake,Cameras,"[Verse:] \r\nWord on road, it's the clique ab..."
2,Queens Of The Stone Age,Into The Void (Black Sabbath Cover),Rocket engines burning fuel so fast \r\nUp in...
3,Il Divo,The Music Of The Night,"Night-time sharpens, heightens each sensation ..."
4,Paul Simon,Still Crazy After All These Years,I met my old lover \r\nOn the street last nig...
5,Peter Cetera,They Don't Make 'em Like They Used To,It's every young boy's fancy \r\nIt's there h...
6,Widespread Panic,Bowlegged Woman,Well take me to my car now \r\nBut the car wo...
7,Usher,Show Me,How's life? \r\nGreat \r\nEverybody's going ...
8,Alabama,Feels So Right,"Whisper to me softly, breathe words upon my sk..."
9,W.A.S.P.,Sweet Cheetah,One night as I rolled down some long winding r...


In [80]:
df['text'][0]

"Out in the west Texas town of El Paso I fell in love with a Mexican girl  \r\nNighttime would find me in Rosa's Cantina music would play and Feleena would\r\nwhirl  \r\nBlacker than night were the eyes of Feleena wicked and evil while casting a\r\nspell  \r\nMy love was deep for this Mexican maiden I was in love but in vain I could\r\ntell  \r\n  \r\nOne night a wild young cowboy came in wild as the west Texas wind  \r\nDashing and daring a drink he was sharing with wicked Feleena the girl that I\r\nloved  \r\nSo in danger I challenged his right for the love of this maiden  \r\nDown went his hand for the gun that he wore  \r\n  \r\nMy challenge was answered in less than a heartbeat  \r\nThe handsome young stranger lay dead on the floor  \r\nJust for a moment I stood there in silence shocked by the foul deed I had done  \r\nMany thoughts raced through my mind as I stood there  \r\n  \r\nI had but one chance and that was to run  \r\nOut to the backdoor of Rosa's I ran out where the hors

In [81]:
df.shape

(5000, 3)

Text Cleaning/Text Preprocessing

In [82]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [49]:
df.tail(5)

,artist,song,text
4995,Grateful Dead,Estimated Prophet,"my time coming, anyday, don't worry about me, ..."
4996,Heart,Cry To Me,poor little dreamer \r stand inside the door ...
4997,Eddie Cochran,Tired And Sleepy,"well i'm a uptown cat, yeah even a crazy pet ..."
4998,David Guetta,Yesterday,"black hearts, black days \r raindrops on my w..."
4999,Stone Roses,I Am The Resurrection,down down you bring me down \r i hear you kno...


In [83]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [84]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)


In [87]:
similarity[0]

array([1.        , 0.02769832, 0.03524887, ..., 0.03649323, 0.03104973,
       0.02880937])

In [88]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [91]:
def recommendation(song_df):
    if song_df not in df['song'].values:
        print(f"{song_df} does not exist in the database.")
        return []

    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [95]:
recommendation('Bang')

['Niggaz Know',
 'Shot Me Down',
 'Bang Bang',
 "Don't Let Go",
 'Myintrotoletuknow',
 "Don't Die",
 "Don't Get Caught",
 'Picture Perfect',
 'Everything',
 'Paper Game',
 "Don't Be Foolish",
 '6 To 14 In 12',
 'Click, Clack - Get Back!',
 "I Can't Swim",
 'Hate',
 'Gz And Hustlas',
 'Get Use To It',
 'Visions Of Home',
 'Last Of A Dying Breed',
 'Bang Bang']

In [96]:
import pickle

In [97]:
pickle.dump(similer, open("similarity", "wb"))

In [98]:
pickle.dump(df, open("df", "wb"))